In [10]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
k = 4
s = -1/(k-1)*30
C_num = 3

In [12]:
file_data = 'bone_data_train_soap_0610.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_x = my_data[:,1:(len(my_data[0,:]))]
n = len(my_x[:,0])
x_dim = len(my_x[0,:])

file_data = 'bone_y_train_soap_0610.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_y = my_data[:,1] - 1 ##注意减一
my_y = my_y.reshape((len(my_y),1))

In [13]:
file_data = 'bone_data_test_soap_0610.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_x_test = my_data[:,1:(len(my_data[0,:]))]
n_test = len(my_x_test[:,0])
x_dim = len(my_x_test[0,:])

file_data = 'bone_y_test_soap_0610.csv'
data = pd.read_csv(file_data)
my_data = data.values

my_y_test = my_data[:,1] - 1 ##注意减一
my_y_test = my_y_test.reshape((len(my_y_test),1))

In [14]:
print(my_x[0,:])

[9.19116308e-01 1.00000000e-05]


In [16]:
my_x

array([[9.19116308e-01, 1.00000000e-05],
       [8.42427019e-01, 1.00000000e-05],
       [1.23667446e+00, 0.00000000e+00],
       [1.02443632e+00, 0.00000000e+00],
       [9.49350329e-01, 0.00000000e+00],
       [1.11130714e+00, 0.00000000e+00],
       [1.08574714e+00, 0.00000000e+00],
       [1.12578748e+00, 0.00000000e+00],
       [9.84561175e-01, 0.00000000e+00],
       [1.08106911e+00, 0.00000000e+00],
       [8.72570861e-01, 1.00000000e-05],
       [9.90524356e-01, 0.00000000e+00],
       [1.08680397e+00, 0.00000000e+00],
       [9.96957345e-01, 1.00000000e-05],
       [8.97532536e-01, 0.00000000e+00],
       [1.11243967e+00, 1.00000000e-05],
       [1.05807403e+00, 0.00000000e+00],
       [9.57510587e-01, 1.00000000e-05],
       [1.07536667e+00, 1.00000000e-05],
       [9.55372670e-01, 1.00000000e-05],
       [8.88381920e-01, 1.00000000e-05],
       [9.06994967e-01, 0.00000000e+00],
       [8.53022500e-01, 0.00000000e+00],
       [1.03596404e+00, 0.00000000e+00],
       [1.115638

In [17]:
my_x = my_x[:,0:1]
my_x_test = my_x_test[:,0:1]
x_dim = 1

In [18]:
my_omega = np.random.randn(x_dim,k)
my_b = np.random.randn(k)
C_num = 0.2
correct_num = 0
iter_num = 200

train_s_cr = []
test_s_cr = []

for i in range(n):
    esti = np.dot(my_x[i,:], my_omega) + my_b
    if esti.argmax() == my_y[i]:
        correct_num += 1
print("初始准确率:",correct_num/n)

for iter in range(iter_num):
    my_beta = np.zeros((n,k))
    ##### beta
    for i in range(n):
        pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
        pred_copy = pred_iter.copy()
        pred_iter[int(my_y[i])] = -float('inf')
        j = pred_iter.argmax()

        if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
            my_beta[i,j] = C_num
    
    ##### function1
    my_alpha = cp.Variable((n,k))
    obj_function = 0
    
    s1 = 0
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

            if my_y[i] != j:
                s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

        s1 += cp.sum_squares(s1_1 - s1_2)
        
    s2 = 0
    for i in range(n):
        for j2 in range(k):
            if j2 != my_y[i]:
                s2 += my_alpha[i,j2]
                
    obj_function = 0.5 * s1 - s2
    
    ##### constrains1
    constrains = []
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

            if my_y[i] != j:
                s1_2 += (my_alpha[i,j] - my_beta[i,j])

        constrains += [(s1_1 - s1_2) == 0]

    for i in range(n):
        s1 = 0
        for j in range(k):
            if j != my_y[i]:
                s1 += my_alpha[i,j]
                constrains += [s1 >= 0]
                constrains += [s1 <= C_num]

    for i in range(n):
        for j in range(k):
            if j != my_y[i]:
                constrains += [my_alpha[i,j] >= 0]
                
    prob = cp.Problem(cp.Minimize(obj_function), constrains)
    prob.solve()
    
    ##### calculate omega
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

            if my_y[i] != j:
                s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

        my_omega[:,j] = s1_1 - s1_2
        
    #print(my_omega)
    
    #####function2
    my_eta = cp.Variable(n)
    my_b = cp.Variable(k)
    
    obj_function_2 = 0
    s1 = 0
    for j in range(k):
        s1_1 = 0
        s1_2 = 0
        for i in range(n):
            if my_y[i] == j:
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s1_1 += my_beta[i,j2]

            if my_y[i] != j:
                s1_2 += my_beta[i,j2]

        s1 += (s1_1 - s1_2) * my_b[j]

    obj_function_2 = C_num * cp.sum(my_eta) + s1
    
    ##### constrains2
    
    constrains_2 = []
    f_sim = np.dot(my_x, my_omega)

    for i in range(n):
        constrains_2 += [my_eta[i] >= 0]

    for i in range(n):
        for j in range(k):
            if j != my_y[i]:
                constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

    constrains_2 += [cp.sum(my_b) == 0]
    
    prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)
    
    prob_2.solve()
    my_b = my_b.value
    #print(my_b)
    
    ### train set correct-ratio ###
    
    correct_num = 0
    esti_mat = np.zeros(n)
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        esti_mat[i] = esti.argmax()
        if esti.argmax() == my_y[i]:
            correct_num += 1
    #print(correct_num/n)
    
    cf_mat = np.zeros((k,k))
    for i in range(n):
        cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
    #print(cf_mat)
    print("iter:{},准确率:{}".format(iter, correct_num/n))
    print(cf_mat)
    train_s_cr.append(correct_num/n)
    
    
    ### test set correct-ratio ###
    correct_num = 0
    esti_mat_test = np.zeros(n_test)
    for i in range(n_test):
        esti = np.dot(my_x_test[i,:], my_omega) + my_b
        esti_mat_test[i] = esti.argmax()
        if esti.argmax() == my_y_test[i]:
            correct_num += 1
    #print(correct_num/n)

    cf_mat = np.zeros((k,k))
    for i in range(n_test):
        cf_mat[int(my_y_test[i]),int(esti_mat_test[i])] += 1
    #print(cf_mat)
    print("准确率:{}".format(correct_num/n_test))
    print(cf_mat)
    test_s_cr.append(correct_num/n_test)

初始准确率: 0.22950819672131148
iter:0,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:1,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:2,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:3,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:4,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 

iter:40,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:41,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:42,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:43,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:44,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1

iter:80,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:81,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:82,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:83,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:84,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1

iter:120,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:121,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:122,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:123,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:124,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]

iter:160,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:161,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:162,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:163,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]
 [ 1.  4.  9.  3.]
 [ 5. 10.  4. 10.]]
iter:164,准确率:0.2786885245901639
[[ 8. 11. 12. 11.]
 [ 5. 22.  8. 10.]
 [ 5.  9. 11. 10.]
 [11. 25. 15. 10.]]
准确率:0.422680412371134
[[ 8.  7.  9.  5.]
 [ 2. 14.  2.  4.]

In [ ]:
x_line = np.linspace(1,iter_num,iter_num)
plt.plot(x_line, train_s_cr)
plt.plot(x_line, test_s_cr)

plt.legend(['train_s_cr','test_s_cr'])
plt.show()